# Классификация и сегментация клиентов iFood

### Постановка задачи

### Компания iFood собирает данные о клиентах и их покупках. Цель — понять поведение клиентов, выделить сегменты (кластеризация) и научиться предсказывать, откликнется ли клиент на маркетинговую кампанию (классификация, целевая переменная - Response).

### Подключение модулей



In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.decomposition import PCA
%matplotlib inline
import warnings
warnings.simplefilter(action='ignore', category=[FutureWarning, UserWarning])
sns.set_style("whitegrid")

In [ ]:
df = pd.read_csv("ifood_df.csv")

In [ ]:
df.head()

# Описание данных
# Числовые признаки:

### Income — годовой доход
### Kidhome, Teenhome — количество детей и подростков
### Recency — дней с последней покупки
### MntWines, MntFruits, MntMeatProducts, ... — суммы трат по категориям
### NumDealsPurchases, NumWebPurchases, ... — количество покупок разными каналами
### NumWebVisitsMonth — посещений сайта за месяц
### AcceptedCmp1–5, AcceptedCmpOverall — принятие предыдущих кампаний
### Age — возраст
### Customer_Days — сколько дней клиент
### MntTotal, MntRegularProds — итоговые траты

# Целевая переменная:

### Response — отклик на последнюю кампанию (0 / 1)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe().round(2)

In [ ]:
plt.figure(figsize=(5, 4))
sns.countplot(x='Response',data=df,palette='viridis',hue='Response',legend=False)
plt.title('Распределение отклика на кампанию (Response)')
plt.xlabel('Response (0 — нет, 1 — да)')
plt.ylabel('Количество клиентов')
plt.show()